# DistilBERT for topic classification
First, load the necessary libraries:

In [1]:
%pip install pyarrow fastparquet


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import json
from collections import Counter
import random
import matplotlib.pyplot as plt

### Preparation and balancing of the dataset

In [3]:
# def load_data(file_path):
#     data = pd.read_parquet(file_path)
#     return data

# def load_pickle_data(file_path):
#     return pd.read_pickle(file_path)

# # Decode the topics_with_percentages column
# def decode_topics(topics):
#     return json.loads(topics.decode('utf-8'))


# def preprocess_data(data):
#     data['topics_with_percentages'] = data['topics_with_percentages'].apply(decode_topics)
#     return data

# def split_data(data):
#     train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
#     return train_df, test_df


# try:
#     from google.colab import drive
#     drive.mount('/content/drive')
#     data_path = '/content/drive/MyDrive/DataScienceLab/GenericMixOfTopic.parquet'
#     pickle_data_path = '/content/drive/MyDrive/DataScienceLab/main_dataset_correctedSingleTopics.pkl'
# except:
#     data_path = 'dsl_project/GenericMixOfTopic.parquet'
#     pickle_data_path = 'dsl_project/main_dataset_correctedSingleTopics.pkl'

# # data = load_data(data_path)
# data = load_pickle_data(pickle_data_path)
# data = data.sample(frac=1).reset_index(drop=True)

# # BALANCING THE DATASET
# # Load the data
# df = data

# def filter_data_for_probability_one(data):
#     # Filter out only rows where at least one label has a probability of 1
#     def has_probability_one(topics):
#         return any(prob == 1 for prob in topics.values())

#     return data[data['topics_with_percentages'].apply(has_probability_one)]



# # Set the desired number of entries
# N = 5000

# # Step 2: Randomly downsample the dataset to have less random samples
# downsampled_df = df.sample(n=N//10, random_state=42)
# downsampled_df.reset_index(drop=True, inplace=True)

# # Step 3: Count the occurrences of non-null probabilities for each label
# def count_non_null_probabilities(df):
#     counts = Counter()
#     for topics in df['topics_with_percentages']:
#         for topic, prob in topics.items():
#             if prob > 0:
#                 counts[topic] += 1
#     return counts

# initial_counts = count_non_null_probabilities(downsampled_df)
# # print("Initial counts of non-null probabilities for each label:")
# # print(initial_counts)

# # Step 4: Take samples with probability of 1 for each label
# def get_samples_with_high_probability(df, label, sample_size):
#     high_prob_samples = df[df['topics_with_percentages'].apply(lambda x: x.get(label, 0) == 1)]
#     return high_prob_samples.sample(n=sample_size, replace=True, random_state=42)

# # Create a new balanced dataframe
# balanced_df = downsampled_df.copy()

# # Calculate the remaining number of samples needed
# remaining_samples_needed = N - len(balanced_df)

# # Distribute the remaining samples among the rare labels first
# rare_labels = [label for label, count in sorted(initial_counts.items(), key=lambda item: item[1])]

# # Calculate the number of samples to be added per label
# label_sample_size = max(remaining_samples_needed // len(rare_labels), 1)

# # Step 5: Take additional samples for rare labels until N entries are reached
# for label in rare_labels:
#     if remaining_samples_needed <= 0:
#         break
#     samples = get_samples_with_high_probability(df, label, min(remaining_samples_needed, label_sample_size))
#     balanced_df = pd.concat([balanced_df, samples])
#     remaining_samples_needed -= len(samples)

# # Ensure we do not exceed N entries
# if len(balanced_df) > N:
#     balanced_df = balanced_df.sample(n=N, random_state=42)
# else:
#     balanced_df = balanced_df.sample(n=len(balanced_df), random_state=42)

# balanced_df.reset_index(drop=True, inplace=True)

# # Verify the counts after balancing
# final_counts = count_non_null_probabilities(balanced_df)

# data = balanced_df

# train_df, test_df = split_data(data)

# def prepare_labels(df):
#     topics = set()
#     for item in df['topics_with_percentages']:
#         topics.update(item.keys())
#     topics = list(topics)

#     label_data = []
#     for item in df['topics_with_percentages']:
#         label_dict = {topic: 0.0 for topic in topics}
#         for topic, percentage in item.items():
#             label_dict[topic] = percentage
#         label_data.append(label_dict)

#     labels_df = pd.DataFrame(label_data)
#     df = df.drop(columns=['topics_with_percentages'])
#     df = pd.concat([df, labels_df], axis=1)

#     return df, topics

# train_df, topics = prepare_labels(train_df)
# test_df, _ = prepare_labels(test_df)
# train_df = train_df.dropna()
# test_df = test_df.dropna()

# # Ensure all label columns are of float type
# label_columns = topics
# train_df[label_columns] = train_df[label_columns].astype(float)
# test_df[label_columns] = test_df[label_columns].astype(float)

In [4]:
def load_pickle_data(file_path):
    return pd.read_pickle(file_path)

def split_data(data):
    train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
    return train_df, test_df

try:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = '/content/drive/MyDrive/DataScienceLab/GenericMixOfTopic.parquet'
    pickle_data_path = '/content/drive/MyDrive/DataScienceLab/main_dataset_correctedSingleTopics.pkl'
except:
    data_path = 'dsl_project/GenericMixOfTopic.parquet'
    pickle_data_path = 'dsl_project/main_dataset_correctedSingleTopics.pkl'
    test_set_path = 'dsl_project/test_set.pkl'

In [5]:
data = load_pickle_data(test_set_path)

# Set the desired number of entries
N = 5000

# Step 2: Randomly downsample the dataset to have less random samples
data = data.sample(n=N//10, random_state=42)

def get_label_columns(df):
    topics = set()
    for item in df['topics_with_percentages']:
        topics.update(item.keys())
    topics = list(topics)
    return topics

label_columns = get_label_columns(data)

In [6]:
# get rows where topic is different from 'Mixed'
data

,id,title,topic,topics_with_percentages,text,Academic_disciplines,Business,Communication,Concepts,Culture,...,People,Philosophy,Politics,Religion,Science,Society,Sports,Technology,Time,Universe
46430,367399,Safrole,Mixed,"{'Science': 0.25, 'Society': 0.25, 'Health': 0...",chembox | Verifiedfields changed | Watchedfi...,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
122070,1904123,Gerard of Csanád,Mixed,"{'People': 0.77, 'Religion': 0.07, 'Society': ...",Gerard or Gerard Sagredo ( ; ; ; 23 April 97...,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
142100,61768376,Pleurotomella protocostulata,Mixed,"{'Life': 0.42, 'Nature': 0.25, 'Health': 0.17,...",'Pleurotomella protocostulata' is an extinct...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11359,34824285,2012 WGC-Accenture Match Play Championship,Mixed,"{'Sports': 0.77, 'Entertainment': 0.1, 'Cultur...",The 2012 WGC-Accenture Match Play Championsh...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
119262,366499,Magnum Research,Mixed,"{'Geography': 0.4, 'Business': 0.27, 'Engineer...",Magnum Research Inc. (MRI) was an American p...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37989,68852442,Border Violence Monitoring Network,Mixed,"{'Government': 0.25, 'History': 0.21, 'Time': ...",The Border Violence Monitoring Network (BVMN...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
113537,22227839,"Hopewell, Washington County, Kansas",History,"{'History': 1.0, 'Academic_disciplines': 0, 'B...",Hopewell is a ghost town in Washington Count...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196256,50548211,Pagus of Hasbania,Mixed,"{'People': 0.5, 'History': 0.33, 'Society': 0....",The pagus or Gau (territory)|gau of 'Hasbani...,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
78057,40050108,Bizjak,Mixed,"{'Language': 0.5, 'Society': 0.5, 'Academic_di...",Bizjak may refer to: Boris Bizjak (born 1981)...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
data.iloc[0]

id                                                                    367399
title                                                                Safrole
topic                                                                  Mixed
topics_with_percentages    {'Science': 0.25, 'Society': 0.25, 'Health': 0...
text                         chembox | Verifiedfields changed | Watchedfi...
Academic_disciplines                                                       0
Business                                                                   0
Communication                                                              0
Concepts                                                                   0
Culture                                                                    0
Economy                                                                    0
Education                                                                  0
Energy                                                                     0

In [8]:
len(label_columns)

39

### LLM Setup

In [9]:
%pip install requests
%pip install backoff
%pip install groq


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
import backoff
import time
import requests
from groq import Groq
import random


class LLM:

    def __init__(self, api_key=None):
        
        self.api_key = api_key
        self.client = Groq(api_key="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")
        model_list = self.client.models.list().to_dict()["data"]

        # remove from momdel_list the entry "whisper-large-v3"
        model_list = [model for model in model_list if model["id"] != "whisper-large-v3"]

        self.models = [
            model["id"] for model in model_list
        ]
        # self.model = "llama3-70b-8192"

    def send_chat_message(self, content) -> str:
        time.sleep(1)
        
        try:
            
            if len(content) > 2 * 8000:
                print("Content too long, skipping...")
                return -1, None
                # updated_model = "mixtral-8x7b-32768"
                # print("Content too long, switching to Mixtral...")

            random_model = random.choice(self.models)

            chat_completion = self.client.chat.completions.create(
                model= random_model,
                messages=[
                    {
                        "role": "user",
                        "content": content,
                    }
                ],
            )

            updated_model = None

            response = chat_completion.choices[0].message.content
            # print(response)
            return response, random_model
        except Exception as e:
            # self.logger.log_error(f"Error sending chat message: {e}")
            print(f"Error sending chat message: {e}")

            # Applying exponential backoff with a maximum of 5 retries
            @backoff.on_exception(
                backoff.expo, requests.exceptions.HTTPError, max_tries=10
            )
            def retry_send_chat_message():
                # self.logger.log_info(f"Rate limit exceeded. Waiting...")
                print("Rate limit exceeded. Waiting...")
                return self.send_chat_message(content)
            return retry_send_chat_message()

In [17]:
llm = LLM()

labels = ','.join(label_columns)

In [18]:
def classify_article(article, labels):
    # classification_prompt = f"""
    #   You are a highly advanced language model specialized in text classification.
    #   Your task is to classify the following article into relevant categories from a predefined list of 39 possible labels.

    #   The labels are: {labels}

    #   When classifying, follow these guidelines:
    #   1. Carefully read the entire article.
    #   2. Assign all relevant labels from the list to the article.
    #   3. If a label is not applicable, do not include it.
    #   4. Ensure the classifications are precise and based on the content.

    #   Here is an example of the format you should follow:

    #   Article:
    #   '''
    #   Julius Julskötare' (&quot;Julius Christmas Handler&quot;) or 'Kalenderhuset' (&quot;The Calendar House&quot;)
    #   was the Sveriges Television's Christmas calendar in 1978. When aired at SVT1|TV1 in 1978, it had no official name,
    #   other than that of Christmas calendar. Plot The main character, played by Björn Skifs , guides the viewers taking
    #   a look at some fictional companies. Video On 23 October 2013, the series was released to DVD . References External
    #   links
    #   '''

    #   Labels: \"Entertainment\", \"Culture\", \"Mass_media\".

    #   Now, classify the following article:

    #   Article:
    #   '''
    #   {article}
    #   '''

    #   Please answer only with at 3 most probable labels separated by commas and without any other text.

    #   Labels:
    #   """
    classification_prompt = f"""
      You are an expert text classifier. Your task is to assign the most relevant labels to the given article from a predefined list of 39 categories.

      Labels: {labels}

      Guidelines:
      1. Thoroughly analyze the article's content.
      2. Assign up to 3 most relevant labels from the provided list.
      3. You may assign fewer than 3 labels if that better represents the content.
      4. Focus on the main themes and topics of the article.
      5. Avoid assigning labels that are only tangentially related.

      Example:
      Article: '''
      Julius Julskötare' ("Julius Christmas Handler") or 'Kalenderhuset' ("The Calendar House") was the Sveriges Television's Christmas calendar in 1978. When aired at SVT1|TV1 in 1978, it had no official name, other than that of Christmas calendar. Plot The main character, played by Björn Skifs, guides the viewers taking a look at some fictional companies. Video On 23 October 2013, the series was released to DVD. References External links
      '''
      Labels: Entertainment, Culture, Mass_media

      Now, classify the following article:

      Article:
      '''
      {article}
      '''

      Provide only the most relevant labels (maximum 3) separated by commas, without any additional text or explanation.

      Labels:
      """

    classification, model = llm.send_chat_message(classification_prompt)
    return classification, len(classification_prompt), model

In [13]:
# classify_article(test_df.iloc[0].text, labels)

NameError: name 'test_df' is not defined

In [19]:
from tqdm import tqdm

small_test_df = data

# Initialize the 'prediction' column with None or any default value
small_test_df['prediction'] = None
small_test_df['model'] = None

In [20]:
for idx in tqdm(small_test_df.index):
  if small_test_df.at[idx, 'prediction']:
    tqdm.write(f"[SKIP] Article {idx+1} already classified")
    continue
  
  article = small_test_df.loc[idx, 'text']
  prediction, len_prediction, model = classify_article(article, labels)
  
  # tqdm.write(f"Article {idx+1}")
  # tqdm.write(f"Prompt length: {len_prediction}")
  # tqdm.write(f"Prediction: {prediction}")

  # Update the 'prediction' column for the current row
  small_test_df.at[idx, 'prediction'] = prediction
  small_test_df.at[idx, 'model'] = model

  0%|          | 1/500 [00:02<21:01,  2.53s/it]

Content too long, skipping...


TypeError: cannot unpack non-iterable int object

In [ ]:
small_test_df

,id,title,topic,topics_with_percentages,text,Academic_disciplines,Business,Communication,Concepts,Culture,...,Philosophy,Politics,Religion,Science,Society,Sports,Technology,Time,Universe,prediction
167321,40041732,"Orchard Park High School, Croydon",Mixed,"{'Education': 0.75, 'Society': 0.25, 'Academic...",Orchard Park High School (formerly Edenham H...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,"Education, History, People \n"
24979,19016500,"Kąty, Gorlice County",Mixed,"{'Geography': 0.5, 'Society': 0.5, 'Academic_d...",Kąty is a village in the administrative dist...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,"Geography, Community, Politics"
39676,533477,List of airports in Turkey,Mixed,"{'Human_behavior': 0.25, 'Economy': 0.25, 'Geo...","This is a list of airport s in Turkey , sort...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Transportation, Infrastructure, Travel"
81759,8584578,Pete Bostwick,Mixed,"{'People': 0.6, 'Sports': 0.26, 'Business': 0....",George Herbert &quot;Pete&quot; Bostwick (Au...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,"Sports, Biography, Horse_racing"
120376,5054048,Sione's Wedding (soundtrack),Mixed,"{'Entertainment': 0.5, 'History': 0.17, 'Time'...",'Sione's Wedding' is the soundtrack to the N...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,"Entertainment, Music, Culture"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95563,26798790,Health Informatics Society of Australia,Mixed,"{'Health': 0.5, 'Society': 0.5, 'Academic_disc...",The Health Informatics Society of Australia ...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,None
31749,12458436,"Ashley, Kent",Society,"{'Society': 1.0, 'Academic_disciplines': 0, 'B...","Ashley (also known, signposted and marked on...",0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,None
138795,21203529,Bieńkowski coat of arms,Concepts,"{'Concepts': 1.0, 'Academic_disciplines': 0, '...",Bieńkowski - is a Polish heraldry|Polish coa...,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,None
158271,66426610,Jeffrey Prescott,Mixed,"{'Government': 0.37, 'People': 0.31, 'Educatio...",Jeffrey Prescott is an American attorney and...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None


In [ ]:
# remove all the rows with value -1 and None in the column "prediction"
mask = (small_test_df["prediction"] != -1) & (small_test_df["prediction"].notna())

clean_small_test_df = small_test_df[mask]

len(clean_small_test_df)

239

In [ ]:
# save the cleaned dataset to a pickle file
import datetime
predictions_path = 'dsl_project/predictions_{date}.pkl'.format(date=datetime.now().strftime('%Y-%m-%d'))

clean_small_test_df.to_pickle(predictions_path)

In [ ]:
# def calculate_accuracy(df):
#     correct = 0
#     total = len(df)
    
#     for _, row in df.iterrows():
#         prediction_list = row['prediction'].split(', ')
        
#         if row['topic'] == 'Mixed':
#             # For Mixed topics, check if columns with 1 are in the prediction list
#             relevant_columns = [col for col in df.columns[5:-1] if row[col] == 1]
#             # print("Is " + str(relevant_columns), "in " + str(prediction_list))
            
#             if any(col in prediction_list for col in relevant_columns):
#                 correct += 1
#         else:
#             # For non-Mixed topics, check if the topic is in the prediction list
#             if row['topic'] in prediction_list:
#                 correct += 1
    
#     accuracy = correct / total
#     return accuracy

# accuracy = calculate_accuracy(clean_small_test_df)
# print(f"Accuracy: {accuracy:.2%}")

In [ ]:
clean_small_test_df.head()

,id,title,topic,topics_with_percentages,text,Academic_disciplines,Business,Communication,Concepts,Culture,...,Philosophy,Politics,Religion,Science,Society,Sports,Technology,Time,Universe,prediction
167321,40041732,"Orchard Park High School, Croydon",Mixed,"{'Education': 0.75, 'Society': 0.25, 'Academic...",Orchard Park High School (formerly Edenham H...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,"Education, History, People \n"
24979,19016500,"Kąty, Gorlice County",Mixed,"{'Geography': 0.5, 'Society': 0.5, 'Academic_d...",Kąty is a village in the administrative dist...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,"Geography, Community, Politics"
39676,533477,List of airports in Turkey,Mixed,"{'Human_behavior': 0.25, 'Economy': 0.25, 'Geo...","This is a list of airport s in Turkey , sort...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Transportation, Infrastructure, Travel"
81759,8584578,Pete Bostwick,Mixed,"{'People': 0.6, 'Sports': 0.26, 'Business': 0....",George Herbert &quot;Pete&quot; Bostwick (Au...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,"Sports, Biography, Horse_racing"
120376,5054048,Sione's Wedding (soundtrack),Mixed,"{'Entertainment': 0.5, 'History': 0.17, 'Time'...",'Sione's Wedding' is the soundtrack to the N...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,"Entertainment, Music, Culture"


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, hamming_loss

def calculate_metrics_improved(df):
    def get_true_labels(row):
        if row['topic'] == 'Mixed':
            return set(str(col) for col in df.columns[5:-1] if row[col] == 1)
        else:
            return {str(row['topic'])}
    
    def get_pred_labels(pred_str):
        return set(label.strip() for label in str(pred_str).split(',') if label.strip())

    y_true = df.apply(get_true_labels, axis=1)
    y_pred = df['prediction'].apply(get_pred_labels)

    correct = sum(len(true.intersection(pred)) > 0 for true, pred in zip(y_true, y_pred))
    total = len(df)

    accuracy = correct / total

    # For other metrics, we'll create binary labels for each unique class
    all_labels = sorted(set.union(*y_true, *y_pred))
    
    y_true_bin = [[1 if label in true_set else 0 for label in all_labels] for true_set in y_true]
    y_pred_bin = [[1 if label in pred_set else 0 for label in all_labels] for pred_set in y_pred]

    f1 = f1_score(y_true_bin, y_pred_bin, average='micro')
    precision = precision_score(y_true_bin, y_pred_bin, average='micro', zero_division=0)
    recall = recall_score(y_true_bin, y_pred_bin, average='micro')
    hamming = hamming_loss(y_true_bin, y_pred_bin)

    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
        'hamming_loss': hamming,
    }

# Use the function
metrics = calculate_metrics_improved(clean_small_test_df)

# Print results
print("LLM Prediction Metrics:")
for metric, value in metrics.items():
    print(f"{metric.capitalize()}: {value:.2%}")

LLM Prediction Metrics:
Accuracy: 72.80%
F1_score: 39.48%
Precision: 34.47%
Recall: 46.18%
Hamming_loss: 2.06%


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import hamming_loss

def calculate_multi_label_accuracies(df):
    def get_true_labels(row):
        if row['topic'] == 'Mixed':
            return set(str(col) for col in df.columns[5:-1] if row[col] == 1)
        else:
            return {str(row['topic'])}
    
    def get_pred_labels(pred_str):
        return set(label.strip() for label in str(pred_str).split(',') if label.strip())

    y_true = df.apply(get_true_labels, axis=1)
    y_pred = df['prediction'].apply(get_pred_labels)


    total_samples = len(df)
    all_labels = sorted(set.union(*y_true, *y_pred))

    # Exact Match Accuracy
    exact_match = sum(t == p for t, p in zip(y_true, y_pred)) / total_samples

    # Subset Accuracy
    subset_accuracy = sum(t.issubset(p) for t, p in zip(y_true, y_pred)) / total_samples

    # Partial Match Accuracy
    partial_match = sum(len(t.intersection(p)) > 0 for t, p in zip(y_true, y_pred)) / total_samples

    # Hamming Accuracy
    y_true_bin = [[1 if label in true_set else 0 for label in all_labels] for true_set in y_true]
    y_pred_bin = [[1 if label in pred_set else 0 for label in all_labels] for pred_set in y_pred]
    
    hamming_accuracy = 1 - hamming_loss(y_true_bin, y_pred_bin)

    # Label-based Accuracy
    label_accuracies = []
    for label in all_labels:
        true_label = [label in true_set for true_set in y_true]
        pred_label = [label in pred_set for pred_set in y_pred]
        label_accuracies.append(sum(t == p for t, p in zip(true_label, pred_label)) / total_samples)
    label_based_accuracy = np.mean(label_accuracies)

    return {
        'exact_match_accuracy': exact_match,
        'subset_accuracy': subset_accuracy,
        'partial_match_accuracy': partial_match,
        'hamming_accuracy': hamming_accuracy,
        'label_based_accuracy': label_based_accuracy
    }

# Use the function
accuracies = calculate_multi_label_accuracies(clean_small_test_df)

# Print results
for metric, value in accuracies.items():
    print(f"{metric.capitalize()}: {value:.2%}")

Exact_match_accuracy: 2.51%
Subset_accuracy: 25.10%
Partial_match_accuracy: 72.80%
Hamming_accuracy: 97.94%
Label_based_accuracy: 97.94%


In [ ]:
def get_true_labels(row):
    if row['topic'] == 'Mixed':
        return set(str(col) for col in df.columns[5:-1] if row[col] == 1)
    else:
        return {str(row['topic'])}
    
def get_pred_labels(pred_str):
    return set(label.strip() for label in str(pred_str).split(',') if label.strip())

df_tmp = clean_small_test_df

y_true = df_tmp.apply(get_true_labels, axis=1)
y_pred = df_tmp['prediction'].apply(get_pred_labels)


total_samples = len(df_tmp)
all_labels = sorted(set.union(*y_true, *y_pred))
all_labels

['Academic_disciplines',
 'Activism',
 'Administrative_divisions',
 'Aesthetics',
 'Animals',
 'Animation',
 'Anime',
 'Architecture',
 'Art',
 'Art_and_Culture',
 'Artists',
 'Arts',
 'Athlete',
 'Aviation',
 'Biography',
 'Botany',
 'Business',
 'Business (for the mention of his professional career)',
 'Chemistry',
 'Chess',
 'City',
 'Clothing',
 'Cognitive_science',
 'Comedy',
 'Communication',
 'Community',
 'Concepts',
 'Culture',
 'Deafness',
 'Economy',
 'Education',
 'Election',
 'Electronics',
 'Energy',
 'Engineering',
 'Entertainment',
 'Entities',
 'Entomology',
 'Environment',
 'Ethics',
 'Film',
 'Fluid_dynamics',
 'Food_and_drink',
 'Foreign_affairs',
 'Furniture',
 'Geography',
 'Glam_metal',
 'Government',
 'Gymnastics',
 'Health',
 'History',
 'Horse_racing',
 'Human-computer_interaction',
 'Human_behavior',
 'Humanities',
 'Information',
 'Information_science',
 'Infrastructure',
 'International_Relations',
 'Italy',
 'Knowledge',
 'Language',
 'Law',
 'Life',
 'Lit